In [1]:
# External dependencies
import os
from time import time

# Get dataframe library - cudf or pandas
from merlin.core.dispatch import get_lib
df_lib = get_lib()

import tritonclient.grpc as grpcclient

In [2]:
# from hugectr.inference import InferenceParams, CreateInferenceSession

In [3]:
import tritonhttpclient

try:
    triton_client = tritonhttpclient.InferenceServerClient(url="10.19.60.80:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


/usr/local/lib/python3.8/dist-packages/tritonhttpclient/__init__.py:31: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
triton_client.is_server_live()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

In [6]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '50'}>
bytearray(b'[{"name":"hps_wdl","version":"1","state":"READY"}]')


[{'name': 'hps_wdl', 'version': '1', 'state': 'READY'}]

In [7]:
%%time
triton_client.load_model(model_name="hps_wdl")

POST /v2/repository/models/hps_wdl/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'hps_wdl'
CPU times: user 397 µs, sys: 467 µs, total: 864 µs
Wall time: 521 µs


In [14]:
!tree /hps_infer/embedding

/hps_infer/embedding
├── hps.json
├── hps_ensemble
│   └── 1
├── hps_torch
│   └── 1
└── hps_wdl
    ├── 1
    │   ├── wdl0_sparse_20000.model
    │   │   ├── emb_vector
    │   │   ├── key
    │   │   ├── wdl0_sparse_20000.model.key
    │   │   └── wdl0_sparse_20000.model.vec
    │   └── wdl1_sparse_20000.model
    │       ├── emb_vector
    │       ├── key
    │       ├── wdl1_sparse_20000.model.key
    │       └── wdl1_sparse_20000.model.vec
    └── config.pbtxt

8 directories, 10 files


In [ ]:
# model_name = 'wdl'
CATEGORICAL_COLUMNS=["C1_C2","C3_C4"]+["C" + str(x) for x in range(1, 27)]
CONTINUOUS_COLUMNS=["I" + str(x) for x in range(1, 14)]
LABEL_COLUMNS = ['label']
# emb_size_array = [278018, 415262,249058, 19561, 14212, 6890, 18592, 4, 6356, 1254, 52, 226170, 80508, 72308, 11, 2169, 7597, 61, 4, 923, 15, 249619, 168974, 243480, 68212, 9169, 75, 34]
# shift = np.insert(np.cumsum(emb_size_array), 0, 0)[:-1]
# test_df=pd.read_csv("/wdl_infer/infer_test.csv",sep=',')

In [ ]:
# read in the workflow (to get input/output schema to call triton with)
batch = df_lib.read_parquet(
    os.path.join(INPUT_DATA_DIR, "part_0.parquet"), num_rows=5, columns=[""]
)
print(batch)